<small><font color=gray>Notebook author: <a href="https://www.linkedin.com/in/olegmelnikov/" target="_blank">Oleg Melnikov</a>, <a href="https://www.hse.ru/en/staff/sara/" target="_blank">Saraa Ali</a>  ©2025 onwards</font></small><hr style="margin:0;background-color:silver">

**[<font size=6>🧬Genomics</font>](https://www.kaggle.com/competitions/25-hse-genomics/rules)**. [**Instructions**](https://colab.research.google.com/drive/1owkYjuRGkx050LQnM3b3yTzd0Dr2XbeV) for running Colabs.

<small>**(Optional) CONSENT.** <mark>[ X ]</mark> We consent to sharing our Colab (after the assignment ends) with other students/instructors for educational purposes. We understand that sharing is optional and this decision will not affect our grade in any way. <font color=gray><i>(If ok with sharing your Colab for educational purposes, leave "X" in the check box.)</i></font></small>

In [ ]:
# from google.colab import drive; drive.mount('/content/drive')   # OK to enable, if your kaggle.json is stored in Google Drive

In [ ]:
!pip -q install --upgrade --force-reinstall --no-deps kaggle > log  # upgrade kaggle package (to avoid a warning)
!mkdir -p ~/.kaggle                               # .kaggle folder must contain kaggle.json for kaggle executable to properly authenticate you to Kaggle.com
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json >log  # First, download kaggle.json from kaggle.com (in Account page) and place it in the root of mounted Google Drive
!cp kaggle.json ~/.kaggle/kaggle.json > log       # Alternative location of kaggle.json (without a connection to Google Drive)
!chmod 600 ~/.kaggle/kaggle.json                  # give only the owner full read/write access to kaggle.json
!kaggle config set -n competition -v 25-hse-genomics # set the competition context for the next few kaggle API calls. !kaggle config view - shows current settings
!kaggle competitions download >> log              # download competition dataset as a zip file
!unzip -o *.zip >> log                            # Kaggle dataset is copied as a single file and needs to be unzipped.
!kaggle competitions leaderboard --show           # print public leaderboard

cp: cannot stat '/content/drive/MyDrive/kaggle.json': No such file or directory
cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method. See setup instructions at https://github.com/Kaggle/kaggle-api/
unzip:  cannot find or open *.zip, *.zip.zip or *.zip.ZIP.

No zipfiles found.
Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method. See setup instructions at https://github.com/Kaggle/kaggle-api/


In [ ]:
!nvidia-smi --query-gpu=gpu_name,memory.total,memory.free,memory.used --format=csv # GPU specs

name, memory.total [MiB], memory.free [MiB], memory.used [MiB]
Tesla T4, 15360 MiB, 15095 MiB, 0 MiB


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%time
%%capture
%reset -f
!pip -q install -U sentence-transformers > log
from IPython.core.interactiveshell import InteractiveShell as IS; IS.ast_node_interactivity = "all"
import numpy as np, pandas as pd, time, matplotlib.pyplot as plt, os, plotly
from sklearn.model_selection import train_test_split
from sentence_transformers import SentenceTransformer as SBERT
from sklearn.svm import SVC, LinearSVC, NuSVC
ToCSV = lambda df, fname: df.round(2).to_csv(f'{fname}.csv', index_label='id') # rounds values to 2 decimals

class Timer():
  def __init__(self, lim:'RunTimeLimit'=60*5): self.t0, self.lim, _ = time.time(), lim, print(f'⏳ started. You have {lim} sec. Good luck!')
  def ShowTime(self):
    msg = f'Runtime is {time.time()-self.t0:.0f} sec'
    print(f'\033[91m\033[1m' + msg + f' > {self.lim} sec limit!!!\033[0m' if (time.time()-self.t0-1) > self.lim else msg)

np.set_printoptions(linewidth=10000, precision=4, edgeitems=20, suppress=True)
pd.set_option('display.max_rows', 100, 'display.max_columns', 100, 'display.max_colwidth', 100, 'display.precision', 2, 'display.max_rows', 4)

CPU times: user 18.9 s, sys: 2.31 s, total: 21.2 s
Wall time: 38 s


In [ ]:
vX = pd.read_csv('testX.csv').set_index('id')
tYX = pd.read_csv('trainYX.csv').set_index('id')
vX

,DNA
id,
100000,TTGATTAATAAGATTCCTTGACACCCTTTGTAAAGTTTCTATTTCGTGTGAAATATCTATCTCTTCAAATCCTTTTAATTTATCTAGGTATTTGCT...
100001,ATTAGTAACGGAGGATTTACTAGATGTTTGGATTTATATTCTAATTTTATTCAGGTGGAAGGGATTGTTTTATGATTCAATAGTATACAGAGAATA...
...,...
119998,CGTCGGCATGCTCGGGCAGTGCGGCGGGCCAGCAGCGTGCCAGTTGTCGCGGGGCGGCCGGGCATCGCGGCGCCGGGCGGCAGCACTCCCGCGAAG...
119999,GCGAGGGCACGAAGGCACGACGGCAACGGCGGCGAGGAGCGCTGTGGCAACCGTCTCCGCGTTTGCGTGCGTACAGCCGAGAGCTGGTTCGCGCAG...


In [ ]:
tmr = Timer() # runtime limit (in seconds). Add all of your code after the timer

⏳ started. You have 300 sec. Good luck!


❗Do not modify the setup above.

<hr color=red>

<font size=5>⏳</font> <strong><font color=orange size=5>Your Code, Documentation, Ideas and Timer - All Start Here...</font></strong>

Students: Keep all your definitions, code, documentation **between** ⏳ symbols.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

train = pd.read_csv("trainYX.csv")
test = pd.read_csv("testX.csv")

id_col = "id"
target_col = "Y"
text_col = "DNA"

X_train_text = train[text_col].astype(str)
y = train[target_col].values
X_test_text = test[text_col].astype(str)
test_ids = test[id_col]

print("[INFO] train shape:", train.shape)
print("[INFO] test  shape:", test.shape)

pos_ratio = y.mean()
print(f"[INFO] Доля класса 1: {pos_ratio:.4f}")


comp_table = str.maketrans({"A": "T", "T": "A", "C": "G", "G": "C"})

def rev_comp(seq: str) -> str:
    return seq[::-1].translate(comp_table)

X_train_rev = X_train_text.apply(rev_comp)

X_train_aug = pd.concat([X_train_text, X_train_rev], ignore_index=True)
y_aug = np.concatenate([y, y])

tfidf = TfidfVectorizer(
    analyzer="char",
    ngram_range=(3, 6),
    min_df=2,
    sublinear_tf=True,
    dtype=np.float32
)

X_train_tfidf = tfidf.fit_transform(X_train_aug)
X_test_tfidf  = tfidf.transform(X_test_text)

use_balanced = (pos_ratio < 0.45) or (pos_ratio > 0.55)
class_weight = "balanced" if use_balanced else None
print(f"[INFO] class_weight = {class_weight}")

Cs = [0.6, 0.7, 0.75, 0.8, 0.9]

skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

best_C = None
best_cv_acc = -1.0

for C in Cs:
    fold_scores = []
    for train_idx, val_idx in skf.split(X_train_tfidf, y_aug):
        X_tr = X_train_tfidf[train_idx]
        X_val = X_train_tfidf[val_idx]
        y_tr = y_aug[train_idx]
        y_val = y_aug[val_idx]

        clf = LinearSVC(
            C=C,
            loss="hinge",
            dual=True,
            class_weight=class_weight,
            random_state=42,
            max_iter=5000,
            tol=1e-4
        )
        clf.fit(X_tr, y_tr)
        val_pred = clf.predict(X_val)
        acc = accuracy_score(y_val, val_pred)
        fold_scores.append(acc)

    mean_acc = np.mean(fold_scores)
    print(f"[CV] C={C} -> mean_acc={mean_acc:.6f}, folds={fold_scores}")

    if mean_acc > best_cv_acc:
        best_cv_acc = mean_acc
        best_C = C

print(f"[INFO] Лучший C по 3-fold CV с аугментацией: {best_C}, cv_acc={best_cv_acc:.6f}")

final_clf = LinearSVC(
    C=best_C,
    loss="hinge",
    dual=True,
    class_weight=class_weight,
    random_state=42,
    max_iter=5000,
    tol=1e-4
)

final_clf.fit(X_train_tfidf, y_aug)
test_pred = final_clf.predict(X_test_tfidf)

submission = pd.DataFrame({
    id_col: test_ids,
    target_col: test_pred
})

submission.to_csv("submission.csv", index=False)
print("[INFO] submission.csv сохранён.")


[INFO] train shape: (100000, 3)
[INFO] test  shape: (20000, 2)
[INFO] Доля класса 1: 0.4994
[INFO] Делаем reverse-complement для train...
[INFO] После аугментации:
       X_train_aug shape: (200000,)
       y_aug shape      : (200000,)
[INFO] Обучаем TF-IDF на аугментированных данных...
[INFO] TF-IDF готов. Форма train: (200000, 5440)
[INFO] class_weight = None
[INFO] Подбираем C через 3-fold CV (loss='hinge', с аугментацией)...


LinearSVC(C=0.6, dual=True, loss='hinge', max_iter=5000, random_state=42)

LinearSVC(C=0.6, dual=True, loss='hinge', max_iter=5000, random_state=42)

LinearSVC(C=0.6, dual=True, loss='hinge', max_iter=5000, random_state=42)

[CV] C=0.6 -> mean_acc=0.983620, folds=[0.9830950845245774, 0.9835300823495883, 0.9842348423484235]


...

LinearSVC(C=0.9, dual=True, loss='hinge', max_iter=5000, random_state=42)

LinearSVC(C=0.9, dual=True, loss='hinge', max_iter=5000, random_state=42)

[CV] C=0.9 -> mean_acc=0.983485, folds=[0.9830650846745767, 0.9833050834745827, 0.9840848408484085]
[INFO] Лучший C по 3-fold CV с аугментацией: 0.6, cv_acc=0.983620
[INFO] Обучаем финальную модель на всех аугментированных данных...


LinearSVC(C=0.6, dual=True, loss='hinge', max_iter=5000, random_state=42)

[INFO] Обучение завершено.
[INFO] Предсказываем test...
[INFO] submission.csv сохранён.


## **Task 1. Preprocessing Pipeline**

Explain elements of your preprocessing pipeline i.e. feature engineering, subsampling, clustering, dimensionality reduction, etc.
1. Why did you choose these elements? (Something in EDA, prior experience,...? Btw, EDA is not required)
1. How do you evaluate the effectiveness of these elements?
1. What else have you tried that worked or didn't?

**Student's answer:**

I (we) use a combination of character-based TF-IDF with a string analyzer, n-grams of length 3–6, and data augmentation by the construction of reverse complementary DNA sequences as the main preprocessing elements. The nature of the data was a basis for choosing char-ngram TF-IDF: the sequences consist of four letters, and their meaning is defined by short patterns (k-mers), rather than words in the usual linguistic sense. N-grams of length 3–6 approximate such k-mers well and allow the model to capture local motifs in DNA. Reverse complementarity reflects the biological symmetry of the double helix: in many problems, the unwound and complementary strands carry the same information, so I (we) double the training set, adding its reverse-complement for each sequence. I (we) also account for possible class imbalance using the class_weight="balanced" parameter in LinearSVC to prevent the model from getting stuck on the majority of classes. I (we) also use a minimum n-gram frequency (min_df=2), which serves as a soft feature selection and reduces noise in the feature matrix.

I (we) evaluate the performance of these elements mainly using stratified k-fold cross-validation on the train set. I (we) split the data into several folds while preserving class proportions, train the LinearSVC model on each train fold, and measure accuracy on the corresponding validation fold, averaging the result across all splits. Using the described approach, I (we) perform the choice of the key hyperparameter C and also compare simultaneously the preprocessing options: with and without augmentation, with different TF-IDF settings. Also, the final model is validated based on the results of a submission to the competition platform: the score in the public part of the test serves as an external check that the chosen pipeline truly generalizes and isn't overfitted via cross-validation.

With the exception of the current pipeline, I (we) tried an approach with universal SBERT text embeddings and the same LinearSVC. In this version, I (we) restricted the training set to the first 10,000 observations and took only the first 50 characters of each sequence to speed up the experiments, but the model trained with these embeddings performed worse: first of all, SBERT is initially trained on natural language and poorly reflects DNA structure; besides, such strong truncation of the sequences results in losing significant information. I (we) have also compared variants without reverse-complement augmentation and without taking class imbalance into account; in such cases, the performance on validation was lower, and the model was more biased toward the leading class. These experiments together brought me (us) to the current preprocessing based on symbolic TF-IDF, reverse-complement augmentation, and C cross-validation tuning as more robust and interpretable.


## **Task 2. Modeling Approach**
Explain your modeling approach, i.e. ideas you tried and why you thought they would be helpful.

1. How did these decisions guide you in modeling?
1. How do you evaluate the effectiveness of these elements?
1. What else have you tried that worked or didn't?

**Student's answer:**

Based on the chosen preprocessing, I (we) was guided by the fact that symbolic TF-IDF produces a very high-dimensional sparse matrix with k-dimensions, essentially a classic "text" formulation. Hence, I (we) have chosen a linear SVM (LinearSVC) with hinge-loss as my (our) main model: it scales well to sparse features, is able to find the maximal separating hyperplane, and has been long proven to be effective in text classification tasks. The decision to use a margin-based model logically follows the idea of augmentation and k-dimensions: I (we) expect that classes may not be perfectly separable in the feature space, and adjusting the C parameter will let me (us) balance the width of the margin with the number of errors, which allows one to tailor the model to the actual complexity of the data. Lastly, class_weight="balanced" for noticeable imbalances prevents the model from learning the trivial "always predict the majority class," and random_state is fixed for reproducibility.

I (we) assess the effectiveness of this approach using stratified cross-validation: I (we) split the training set into several folds while saving class proportions. At each step, I (we) train LinearSVC on the train set and measure accuracy on the validation set. Then, I (we) average the results across all folds. Within this scheme, I (we) iterate over the values of the parameter C, compare variants with and without augmentation, with and without taking imbalance into account, and choose the best combination that gives a sufficiently stable result across folds. Having chosen a configuration, I (we) further train the model on the entire augmented train set and validate it in a hidden test via competition submissions. If the cross-validation score and the public score on the lead are consistent, this serves as additional evidence that the chosen modeling approach is truly generalizable and not simply tailored to a specific partition.

Among the alternative ideas, I (we) tried using a more "universal" stack: getting sequence embeddings with the pre-trained SBERT language model and training LinearSVC in this embedding space. In practice, this approach turned out to be weaker: firstly, because of the embedding model itself being trained on natural language, not DNA; secondly, due to limitations of time and memory, I (we) had to take only the first 10,000 examples and truncate sequences to 50 characters, by which quality was degraded further. I (we) also tried changing the settings of LinearSVC within the chosen pipeline—changing the C range and disabling augmentation or class_weight—and observed that the model either overfitted or became more biased toward the dominant class. These unsuccessful attempts and comparative experiments reinforced my (our) choice of the current approach: a linear SVM on top of TF-IDF k-mers with biologically motivated augmentation and careful regularization tuning.

# **References:**

1. Remember to cite your sources here as well! At the least, your textbook should be cited. Google Scholar allows you to effortlessly copy/paste an APA citation format for books and publications. Also cite StackOverflow, package documentation, and other meaningful internet resources to help your peers learn from these (and to avoid plagiarism claims).
1. Starter Ideas from this file
1. ...

<font color=green><h4><b>$\epsilon$. LLM Documentation if used</b></h4></font>

This time I(we) did without the help of blue whales, or deepseek, but chatGPT, as always, came to my(our) rescue with an explanation of the initial code, and also helped me(us) understand the starting ideas, describing in detail what each of them would bring to our code.

<font size=5>⌛</font> <strong><font color=orange size=5>Do not exceed competition's runtime limit!</font></strong>

<hr color=red>


In [ ]:
tmr.ShowTime()    # measure Colab's runtime. Do not remove. Keep as the last cell in your notebook.

Runtime is 144 sec


# 💡**Starter Ideas**

1. Learn about DNA [&#127910;](https://www.youtube.com/results?search_query=nucleotides+genes+amino+acids+)
1. Try a larger training sample.
1. Try longer training DNA strings, but SBERT may have a cap on string length, so you might split DNA into several strings and then concatenate or average resulting vectors
1. Try other pretrained SBERT models. Note that DNA sequence uses ACGT letters, but many other models were trained on multilingual text. So, you might prefer those that were trained on mostly ASCII.
1. SBERT is trained on word tokens (typically, separated by spaces), but DNA sequence has no spaces. Try placing spaces after every character or some semantically meaningful subsequences (this might require more domain knowledge).
1. Try Google's [USE](https://tfhub.dev/google/universal-sentence-encoder-multilingual/3) embedding models
1. Try Facebook's [LASER](https://github.com/facebookresearch/LASER) and [others](https://tfhub.dev/s?module-type=text-language-model).
1. Try [Enformer](https://tfhub.dev/deepmind/enformer/1) for gene expressions. See [DeepMind paper](https://deepmind.com/blog/article/enformer).
1. Try building your own embeddings on the given sequences. SBERT and other packages make it easy (just a few lines), but it may take too much time.
1. Assess distribution of character patterns (single, doubles, triplets, ...). For example, an ACGT string generates AC, CG, GT doubles and ACG and CGT triplets. Does one class have more subsequences of some type? This might be a feature in your model.
1. Try features built as counts of subsequences (singles, doubles, triplets, ...). Consider EDA first.
1. Concatenate or otherwise combine multiple embeddings derived from each gene string
1. Learn from [*The genetic code*](https://www.khanacademy.org/science/ap-biology/gene-expression-and-regulation/translation/a/the-genetic-code-discovery-and-properties), Khan Academy.
1. Learn from [*Apply Machine Learning Algorithms for Genomics Data Classification*](https://medium.com/mlearning-ai/apply-machine-learning-algorithms-for-genomics-data-classification-132972933723)
1. Learn from [*Efficient counting of k-mers in DNA sequences using a bloom filter*](https://bmcbioinformatics.biomedcentral.com/articles/10.1186/1471-2105-12-333) Páll Melsted et al. 2011
1. Try [Byte Pair Encoding](https://www.derczynski.com/papers/archive/BPE_Gage.pdf) and [SentencePiece](https://arxiv.org/pdf/1808.06226.pdf) to auto identification of "important" [k-mers](https://en.wikipedia.org/wiki/K-mer) (substrings)